<a href="https://colab.research.google.com/github/Lnguyen19/spam_detections/blob/main/spamDectection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"lam8134","key":"154eb2b43b31cd43cec727b1bf231669"}'}

In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d uciml/sms-spam-collection-dataset

100% 211k/211k [00:00<00:00, 386kB/s]
100% 211k/211k [00:00<00:00, 386kB/s]


In [4]:
!unzip sms-spam-collection-dataset.zip

Archive:  sms-spam-collection-dataset.zip
  inflating: spam.csv                


In [5]:
!pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.5 MB/s eta 0:00:00


In [6]:
import keras
from keras.datasets import imdb
from keras_preprocessing.sequence import pad_sequences
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Embedding, SpatialDropout1D, Conv1D, GlobalMaxPool1D, Conv1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer

from sklearn.preprocessing import LabelEncoder
%matplotlib inline

from keras.layers import SimpleRNN

In [7]:
data = pd.read_csv('spam.csv', encoding='latin-1')
print(data.head())
message = data['v2'].values
identifier = data['v1'].values


     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


In [8]:

n_rnn = 256
drop_rnn = 0.2
drop_embed = 0.2

In [9]:
epochs = 12
batch_size = 128

n_dim = 100
n_unique_words = 5000
total_elements = 5405
n_words_to_skip = 50
pad_type = trunc_type = 'pre'
n_dense = 100
dropout = 0.4

In [24]:
max_review_length = 1000
max_len = 300
token = Tokenizer(num_words= max_review_length)
token.fit_on_texts(message)
sequences = token.texts_to_sequences(message)
x = pad_sequences(sequences, maxlen=max_len)
encoder = LabelEncoder()
y = encoder.fit_transform(identifier)
x_train, x_test, y_train, y_test = train_test_split(x , y, test_size=0.2, random_state=42)


In [37]:

model = Sequential()
model.add(Embedding(n_unique_words, n_dim, input_length=max_len))
model.add(SpatialDropout1D(drop_embed))
model.add(SimpleRNN(n_rnn, dropout=drop_rnn, return_sequences=True))  # Stacked SimpleRNN layers
model.add(SimpleRNN(n_rnn, dropout=drop_rnn, return_sequences=True))
model.add(SimpleRNN(n_rnn, dropout=drop_rnn))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history =  model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))


Epoch 1/12
35/35 [==============================] - 37s 939ms/step - loss: 0.4271 - accuracy: 0.8517 - val_loss: 0.3967 - val_accuracy: 0.8655
Epoch 2/12
35/35 [==============================] - 33s 933ms/step - loss: 0.3977 - accuracy: 0.8661 - val_loss: 0.3983 - val_accuracy: 0.8655
Epoch 3/12
35/35 [==============================] - 30s 868ms/step - loss: 0.3949 - accuracy: 0.8661 - val_loss: 0.4074 - val_accuracy: 0.8655
Epoch 4/12
35/35 [==============================] - 31s 879ms/step - loss: 0.3973 - accuracy: 0.8661 - val_loss: 0.3964 - val_accuracy: 0.8655
Epoch 5/12
35/35 [==============================] - 31s 870ms/step - loss: 0.3947 - accuracy: 0.8661 - val_loss: 0.3940 - val_accuracy: 0.8655
Epoch 6/12
35/35 [==============================] - 31s 876ms/step - loss: 0.3938 - accuracy: 0.8661 - val_loss: 0.3942 - val_accuracy: 0.8655
Epoch 7/12
35/35 [==============================] - 30s 851ms/step - loss: 0.4089 - accuracy: 0.8569 - val_loss: 0.3911 - val_accuracy: 0.8655

In [38]:
def prediction(text):
  sequence = token.texts_to_sequences([text])
  pad = pad_sequences(sequence, maxlen=max_len)
  predict_ = model.predict(pad)
  return predict_[0,0]


In [41]:
prob = prediction("Hurry! Last chance to double your money. Invest now for guaranteed returns.")
if(prob > 0.5):
  print(prob)
  print('Spam')
else:
  print(prob)
  print('Not Spam')


1/1 [==============================] - 0s 96ms/step
0.90932316
Spam
